# Stimulus Coding Example

In this tutorial we illustrate how to use the regression approach to model the effect of stimulus coding on the drift rate parameter of the DDM.

### Import Modules

In [1]:
from copy import deepcopy
import arviz as az
import pandas as pd
import hssm

## What is Stimulus Coding?

There are two core approaches to coding the stimuli when fitting paramters of 2-choice SSMs (the discussion here is simplified, to bring across the core ideas, de facto ideas from both approaches can be mixed):

1. *Accuracy coding*: Responses are treated as **correct** or **incorrect**
2. *Stimulus coding*: Responses are treated as **stimulus_1** or **stimulus_2**

Take as a running example a simple random dot motion task with two conditions, `left` and `right`. Both conditions are equally *difficult*, but for half of the experiments the correct motion direction is left, and for the other half it is right.

So it will be reasonable to assume that, ceteris paribus, nothing should really change in terms of participant behavior, apart from symmetrically preferring right to left when it is correct and vice versa. 

Now, when applying *Accuracy coding*, we would expect the drift rate to be the same for both conditions, any condition effect to vanish by the time we code responses as correct or incorrect.

When we apply *Stimulus coding* on the other hand, we actively need to account for the direction change (since we now attach our *response values*, e.g. `-1`, `1`, permanently to specific choice-options, regardless correctness). 

To formulate a model that is equivalent to the one described above in terms of *accuracy coding*, we again want to estimate only a single `v` parameter, but we have to respect the direction change in response when respectively completing experiment conditions `left` and `right`.

Note that an important aspect of what we describe above is that we want to estimate a single `v` parameter in each of the two *coding approaches*.

For *Accuracy coding* we simply estimate a single `v` parameter, and no extra work is necessary.

For *Stimulus coding* we need to account for **symmetric** shift in direction from the two experiment conditions. One way to do this, is the following:

We can simply assign a covariate, `direction`, which codes `-1` for `left` and `1` for `right`.
Then we use the following regression formula for the `v` parameter: `v ~ 0 + direction`. 

Note that we are *not using an intercept* here.

Let's how this works in practice.

## Simulate Data


In [2]:
# Condition 1
stim_1 = hssm.simulate_data(
    model="ddm", theta=dict(v=-0.5, a=1.5, z=0.5, t=0.1), size=500
)

stim_1["stim"] = "C-left"
stim_1["direction"] = -1
stim_1["response_acc"] = (-1) * stim_1["response"]

# Condition 2
stim_2 = hssm.simulate_data(
    model="ddm", theta=dict(v=0.5, a=1.5, z=0.5, t=0.1), size=500
)

stim_2["stim"] = "C-right"
stim_2["direction"] = 1
stim_2["response_acc"] = stim_2["response"]

data_stim = pd.concat([stim_1, stim_2]).reset_index(drop=True)

data_acc = deepcopy(data_stim)
data_acc["response"] = data_acc["response_acc"]

print(data_acc.head())
print(data_stim.head())

         rt  response    stim  direction  response_acc
0  0.765660       1.0  C-left         -1           1.0
1  2.073602       1.0  C-left         -1           1.0
2  2.756536      -1.0  C-left         -1          -1.0
3  0.421932       1.0  C-left         -1           1.0
4  3.323014       1.0  C-left         -1           1.0
         rt  response    stim  direction  response_acc
0  0.765660      -1.0  C-left         -1           1.0
1  2.073602      -1.0  C-left         -1           1.0
2  2.756536       1.0  C-left         -1          -1.0
3  0.421932      -1.0  C-left         -1           1.0
4  3.323014      -1.0  C-left         -1           1.0


## Set up Models

### Accuracy Coding


In [3]:
m_acc_stim_dummy = hssm.HSSM(
    data=data_acc,
    model="ddm",
    include=[{"name": "v", "formula": "v ~ 1 + stim"}],
    z=0.5,
)

m_acc_stim_dummy.sample(sampler="mcmc", tune=500, draws=500)

m_acc_stim_dummy.summary()

Model initialized successfully.
Using default initvals. 



Initializing NUTS using adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [a, t, v_Intercept, v_stim]


Output()

Sampling 4 chains for 500 tune and 500 draw iterations (2_000 + 2_000 draws total) took 7 seconds.
100%|██████████| 2000/2000 [00:01<00:00, 1139.43it/s]


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
a,1.418,0.026,1.372,1.467,0.001,0.001,1649.0,1498.0,1.0
t,0.146,0.017,0.115,0.175,0.000,0.000,1378.0,1071.0,1.0
v_stim[C-right],0.061,0.050,-0.034,0.153,0.001,0.001,1830.0,1570.0,1.0
v_Intercept,0.503,0.036,0.435,0.570,0.001,0.001,1877.0,1680.0,1.0


In [4]:
m_acc_simple = hssm.HSSM(
    data=data_acc,
    model="ddm",
    include=[
        {
            "name": "v",
            "formula": "v ~ 1",
            "prior": {"Intercept": {"name": "Normal", "mu": 0.0, "sigma": 3.0}},
        }
    ],
    z=0.5,
)

m_acc_simple.sample(sampler="mcmc", tune=500, draws=500)

m_acc_simple.summary()

Model initialized successfully.
Using default initvals. 



Initializing NUTS using adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [a, t, v_Intercept]


Output()

Sampling 4 chains for 500 tune and 500 draw iterations (2_000 + 2_000 draws total) took 8 seconds.
100%|██████████| 2000/2000 [00:01<00:00, 1776.00it/s]


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
a,1.417,0.026,1.366,1.463,0.001,0.001,1286.0,1040.0,1.0
t,0.146,0.017,0.114,0.177,0.000,0.000,1353.0,1210.0,1.0
v_Intercept,0.533,0.027,0.480,0.581,0.001,0.001,1377.0,1260.0,1.0


In [5]:
az.compare({"m_acc_simple": m_acc_simple.traces, 
            "m_acc_stim_dummy": m_acc_stim_dummy.traces})

,rank,elpd_loo,p_loo,elpd_diff,weight,se,dse,warning,scale
m_acc_simple,0,-1934.137814,2.890788,0.000000,0.68423,33.537324,0.000000,False,log
m_acc_stim_dummy,1,-1934.384055,3.830290,0.246242,0.31577,33.546728,1.155514,False,log


## Stim Coding

In [6]:
m_stim = hssm.HSSM(
    data=data_stim,
    model="ddm",
    include=[
        {
            "name": "v",
            "formula": "v ~ 0 + direction",
            "prior": {"direction": {"name": "Normal", "mu": 0.0, "sigma": 3.0}},
        }
    ],
    z=0.5,
)

m_stim.sample(sampler="mcmc", tune=500, draws=500)

m_stim.summary()

Model initialized successfully.
Using default initvals. 



Initializing NUTS using adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [a, t, v_direction]


Output()

Sampling 4 chains for 500 tune and 500 draw iterations (2_000 + 2_000 draws total) took 6 seconds.
100%|██████████| 2000/2000 [00:00<00:00, 2677.81it/s]


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
a,1.416,0.027,1.368,1.468,0.001,0.001,1313.0,1262.0,1.0
t,0.147,0.017,0.115,0.177,0.001,0.001,1152.0,925.0,1.0
v_direction,0.532,0.027,0.485,0.583,0.001,0.001,1216.0,1194.0,1.0


In [7]:
az.compare(
    {
        "m_acc_simple": m_acc_simple.traces,
        "m_acc_stim_dummy": m_acc_stim_dummy.traces,
        "m_stim": m_stim.traces,
    }
)

,rank,elpd_loo,p_loo,elpd_diff,weight,se,dse,warning,scale
m_acc_simple,0,-1934.137814,2.890788,0.000000,6.841569e-01,33.537324,0.000000,False,log
m_stim,1,-1934.223498,2.977101,0.085684,1.479114e-31,33.507170,0.045821,False,log
m_acc_stim_dummy,2,-1934.384055,3.830290,0.246242,3.158431e-01,33.546728,1.155514,False,log


## Stim coding advanced

So far we focused on the `v` parameter. The are two relevant concepts concerning `bias` that we need to account for in the *stimulus coding* approach: 

#### 1. Bias in `v`:

What is drift bias? Imagine our experimental design is such that the correct motion direction is left for half of the experiments and right for the other half. However, the sensory stimuli are such that the participant will nevertheless be accumulating excess evidence toward the left direction, even when the correct motion direction is right for a given trial.
To account for drift bias, we simply include an `Intercept` term, which will capture the drift bias, so that the `direction` term will capture the *direction effect*, a symmetric shift around the `Intercept` (previously this `Intercept` was set to 0, or appeared in the model that operated on a dummy `stim` variable, which remember, creates a models that is too complex / has unnecessary extra parameters).

 #### 2. Bias in `z`:

Bias in the `z` parameter gets a bit more tricky. What's the idea here? The `z` parameter represents the *starting point bias*. This notion is to some extend more intuitive when using *stimulus coding* than *accuracy coding*. A starting point bias under the stimulus coding approach is a bias toward a specific choice option (direction). A starting point bias under the accuracy coding approach is a ... bias toward a correct or incorrect response ... (?)

By itself not a problem, but to create the often desired symmetry in the `z` parameter across `stim` conditions, keeping in mind that bias takes values in the interval `[0, 1]`, we need to account for the direction effect in the `z` parameter. So if in the `left` condition $z_i = z$, then in the `right` condition $z_i = 1 - z$.

How might we incoporate this into our regression framework?

Consider the following varible $\mathbb{1}_{C_i = c}, \text{for} \ c \in \{left, right\}$ which is 1 if the condition is `left` and 0 otherwise for a given trial. Now we can write the following function for $z_i$,


$$  z_i = \mathbb{1}_{(C_i = left)} \cdot z + (1 - \mathbb{1}_{(C_i = left)}) \cdot (1 - z) $$

which after a bit of algebra can be rewritten as,

$$ z_i = \left((2 \cdot \mathbb{1}_{(C_i = left)}) - 1\right) \cdot z + (1 - \mathbb{1}_{(C_i = left)}) $$

or,

$$ z_i = \left((2 \cdot \mathbb{1}_{(C_i = left)}) - 1\right) \cdot z + \mathbb{1}_{(C_i = right)} $$

This is a linear function of the `z` parameter, so we will be able to include it in our model, with a little bit of care.

You will see the use of the `offset` function, to account for the `right` condition, and we will a priori massage our data a little to define the `left.stimcoding` and `right.stimcoding` covariates (dummy variables that identify the `left` and `right` conditions). 

### Defining the new covariates

In [8]:
# Folling the math above, we can define the new covariates as follows:
data_stim["left.stimcoding"] = (2 * (data_stim["stim"] == "C-left").astype(int)) - 1
data_stim["right.stimcoding"] = (data_stim["stim"] == "C-right").astype(int)

### Defining the model

Below an example of a model that take into account both the bias in `v` and in `z`.

In [9]:
m_stim_inc_z = hssm.HSSM(
    data=data_stim,
    model="ddm",
    include=[
        {
            "name": "v",
            "formula": "v ~ 0 + direction",
            "prior": {"direction": {"name": "Normal", "mu": 0.0, "sigma": 3.0}},
        },
        {
            "name": "z",
            "formula": "z ~ 0 + left.stimcoding + offset(right.stimcoding)",
            "prior": {
                "left.stimcoding": {"name": "Uniform", "lower": 0.0, "upper": 1.0},
            },
        },
    ],
)

m_stim_inc_z.sample(sampler="mcmc", tune=500, draws=500)

m_stim_inc_z.summary()

Model initialized successfully.
Using default initvals. 



Initializing NUTS using adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [a, t, v_direction, z_left.stimcoding]


Output()

Sampling 4 chains for 500 tune and 500 draw iterations (2_000 + 2_000 draws total) took 11 seconds.
100%|██████████| 2000/2000 [00:00<00:00, 2560.91it/s]


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
a,1.417,0.026,1.373,1.467,0.001,0.001,1303.0,1402.0,1.0
t,0.146,0.019,0.109,0.181,0.001,0.000,1263.0,1065.0,1.0
z_left.stimcoding,0.500,0.013,0.475,0.523,0.000,0.000,1225.0,1130.0,1.0
v_direction,0.532,0.034,0.469,0.595,0.001,0.001,1205.0,1349.0,1.0


In [10]:
az.compare(
    {
        "m_acc_simple": m_acc_simple.traces,
        "m_acc_stim_dummy": m_acc_stim_dummy.traces,
        "m_stim": m_stim.traces,
        "m_stim_inc_z": m_stim_inc_z.traces,
    }
)

,rank,elpd_loo,p_loo,elpd_diff,weight,se,dse,warning,scale
m_acc_simple,0,-1934.137814,2.890788,0.000000,0.684171,33.537324,0.000000,False,log
m_stim,1,-1934.223498,2.977101,0.085684,0.000000,33.507170,0.045821,False,log
m_acc_stim_dummy,2,-1934.384055,3.830290,0.246242,0.315829,33.546728,1.155514,False,log
m_stim_inc_z,3,-1935.079805,3.749734,0.941991,0.000000,33.535745,0.096578,False,log
